In [1]:
import lstm

We will start by defining a simple wrapper class that contains the entire parametrization of the  LSTM network.
This is implemented in `lstm.Parameters`.

In [2]:
%psource lstm.Parameters

class Parameters:
    def __init__(self, event_size: int, hidden_size: int):
        # Weights and biases for the sigmoid "f-function"
        self.event_forget_weights = np.zeros((hidden_size, event_size))
        self.event_forget_bias = 0
        self.hidden_forget_weights = np.zeros((hidden_size, hidden_size))
        self.hidden_forget_bias = 0

        # Weights and biases for the sigmoid "i-function"
        self.event_update_weights = np.zeros((hidden_size, event_size))
        self.event_update_bias = 0
        self.hidden_update_weights = np.zeros((hidden_size, hidden_size))
        self.hidden_update_bias = 0

        # Weights and biases for the tanh "C-bar-function"
        self.event_candidate_weights = np.zeros((hidden_size, event_size))
        self.event_candidate_bias = 0
        self.hidden_candidate_weights = np.zeros((hidden_size, hidden_size))
        self.hidden_candidate_bias = 0

        # Weights and biases for the "o-function"
        self.event_output_weight

We will now instantiate this class.
For our dummy example, we will set the input size to 3 and the hidden size to 3 as well.

In [19]:
EVENT_SIZE, HIDDEN_SIZE = 3, 3
parameters = lstm.Parameters(event_size=EVENT_SIZE, hidden_size=HIDDEN_SIZE)

The forget gate is implemented as follows:

In [20]:
%psource lstm.forget_gate

def forget_gate(event, hidden_state, prev_cell_state, parameters):
    """Forget gate deciding how much of the previous cell state to keep."""
    forget_hidden = (
        parameters.hidden_forget_weights @ hidden_state
        + parameters.hidden_forget_bias
    )
    forget_event = (
        parameters.event_forget_weights @ event
        + parameters.event_forget_bias
    )
    # Values between zero and one indicating how much to forget
    forgetter = sigmoid(forget_hidden + forget_event)

    # Return the state that should be kept
    kept_state = forgetter * prev_cell_state
    return kept_state


Let's assume that we have an existing hidden state of $\vec{h} = [0, 0, 10]$, a previous cell state of $\vec{C} = [10, 10, 10]$, and a new event $\vec{x} = [10, 0, 0]$.
What happens in the forget gate with the current parametrization?

In [21]:
import numpy as np

event = np.array([10, 0, 0])
hidden_state = np.array([0, 0, 10])
prev_cell_state = np.array([1, 1, 1])
lstm.forget_gate(event=event, hidden_state=hidden_state, parameters=parameters, prev_cell_state=prev_cell_state)

array([0.5, 0.5, 0.5])

Since we only have zero-weights, and zero biases, the new cell state has become $[0.5, 0.5, 0.5]$ since $\mathrm{sigmoid}(1) = 0.5$.
Let's change this:

In [22]:
parameters.event_forget_weights = np.eye(HIDDEN_SIZE, EVENT_SIZE)
parameters.hidden_forget_weights = np.eye(HIDDEN_SIZE, EVENT_SIZE)
lstm.forget_gate(event=event, hidden_state=hidden_state, parameters=parameters, prev_cell_state=prev_cell_state)

array([0.9999546, 0.5      , 0.9999546])

Changing both weight matrices to the identity matrices, the previous hidden state and the new event has influenced the new cell state.
We can also let the new event influence the cell state, and the hidden state be ignored completely:

In [24]:
parameters.event_forget_weights = np.eye(HIDDEN_SIZE, EVENT_SIZE)
parameters.hidden_forget_weights = np.zeros((HIDDEN_SIZE, EVENT_SIZE))
lstm.forget_gate(event=event, hidden_state=hidden_state, parameters=parameters, prev_cell_state=prev_cell_state)

array([0.9999546, 0.5      , 0.5      ])

---

The 